<a href="https://colab.research.google.com/github/Prasanna-03/-LGMVIP--DataScience/blob/main/nwp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [15]:
from google.colab import files
uploaded = files.upload()

Saving 1661-0.txt to 1661-0 (1).txt


In [16]:
file = open("1661-0.txt", "r", encoding = "utf8")
lines =[]
for i in file:
	lines.append(i)
data = ""
for i in lines:
	data = ' '.join(lines)
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('"', '').replace('"', '')
data = data.split()
data = ' '.join(data)
data[:500]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.net Title: The Adventures of Sherlock Holmes Author: Arthur Conan Doyle Release Date: November 29, 2002 [EBook #1661] Last Updated: May 20, 2019 Language: English Character set en"

In [17]:
len(data)

578728

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer, open('token.pkl', 'wb'))
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]


[145, 4789, 1, 1020, 4, 128, 34, 45, 611, 2235, 2236, 30, 1021, 15, 23]

In [19]:
len(sequence_data)

111252

In [20]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)


8931


In [21]:
sequences = []
for i in range(3, len(sequence_data)):
	words = sequence_data[i-3:i+1]
	sequences.append(words)
print("The length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The length of sequences are:  111249


array([[ 145, 4789,    1, 1020],
       [4789,    1, 1020,    4],
       [   1, 1020,    4,  128],
       [1020,    4,  128,   34],
       [   4,  128,   34,   45],
       [ 128,   34,   45,  611],
       [  34,   45,  611, 2235],
       [  45,  611, 2235, 2236],
       [ 611, 2235, 2236,   30],
       [2235, 2236,   30, 1021]])

In [22]:
X = []
y = []
for i in sequences:
	X.append(i[0:3])
	y.append(i[3])
X = np.array(X)
y = np.array(y)


In [23]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[ 145 4789    1]
 [4789    1 1020]
 [   1 1020    4]
 [1020    4  128]
 [   4  128   34]
 [ 128   34   45]
 [  34   45  611]
 [  45  611 2235]
 [ 611 2235 2236]
 [2235 2236   30]]
Response:  [1020    4  128   34   45  611 2235 2236   30 1021]


In [24]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [25]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             89310     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8931)              8939931   
                                                                 
Total params: 22,078,241
Trainable params: 22,078,241
Non-trainable params: 0
_________________________________________________________________


In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=20, batch_size=228, callbacks=[checkpoint])

Epoch 1/20
488/488 [==============================] - ETA: 0s - loss: 6.4328
Epoch 1: loss improved from inf to 6.43284, saving model to next_words.h5
488/488 [==============================] - 706s 1s/step - loss: 6.4328
Epoch 2/20
488/488 [==============================] - ETA: 0s - loss: 6.0991
Epoch 2: loss improved from 6.43284 to 6.09911, saving model to next_words.h5
488/488 [==============================] - 667s 1s/step - loss: 6.0991
Epoch 3/20
488/488 [==============================] - ETA: 0s - loss: 5.7491
Epoch 3: loss improved from 6.09911 to 5.74908, saving model to next_words.h5
488/488 [==============================] - 660s 1s/step - loss: 5.7491
Epoch 4/20
488/488 [==============================] - ETA: 0s - loss: 5.4833
Epoch 4: loss improved from 5.74908 to 5.48332, saving model to next_words.h5
488/488 [==============================] - 660s 1s/step - loss: 5.4833
Epoch 5/20
488/488 [==============================] - ETA: 0s - loss: 5.2573
Epoch 5: loss improved 

In [29]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle


model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
	sequence = tokenizer.texts_to_sequences([text])
	sequence = np.array(sequence)
	preds = np.argmax(model.predict(sequence))
	predicted_word = ""
	
	for key, value in tokenizer.word_index.items():
		if value == preds:
			predicted_word = key
			break
	print(predicted_word)
	return predicted_word



In [30]:
while(True):
	text = input("Enter your line: ")
	if text == "0":
		print("Execution completed.....")
		break
	else:
		try:
			text = text.split(" ")
			text = text[-3:]
			print(text)
			Predict_Next_Words(model, tokenizer, text)
		except Exception as e:
			print("Error occurred: ",e)
			continue

Enter your line: with this eBook
['with', 'this', 'eBook']
or
Enter your line: The Adventures of
['The', 'Adventures', 'of']
sherlock
Enter your line: 0
Execution completed.....
